# Predicting heart disease using machine learning

This notebook looks into using various Python based machine learning and data science libraries in an an attempt to build a machine learning model capable of predicitng whether someone has heart disease or not based on their medical attributes.